In [ ]:
import sys
import os

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))  # Adjust the path accordingly

In [ ]:
import pandas as pd
import numpy as np
from viterbi import Predictor
from load_data import *
import pickle

In [ ]:
data_file = "../data/brown-universal.txt"
tags_file = "../data/tags-universal.txt"
NLTK_model = "../results/hmm_tagger-NLTK.pkl"
BW_model = "../results/hmm_tagger-SS.pkl" 

# Make Predictions

With NLTK Trained HMM

In [ ]:
with open(NLTK_model, 'rb') as pickle_file:
    hmm_tagger_NLTK = pickle.load(pickle_file)

In [ ]:
# Extract model parameters
tags_nltk = hmm_tagger_NLTK._states  # Set of all possible tags
words = hmm_tagger_NLTK._symbols  # Set of all possible words

# convert probability distributions of HMM to dictionaries
transitions = {}
for prev_state in hmm_tagger_NLTK._transitions:
    transitions[prev_state] = {}
    for next_state in tags_nltk:
        transitions[prev_state][next_state] = hmm_tagger_NLTK._transitions[prev_state].prob(next_state)

emissions = {}
for state in tags_nltk:
    emissions[state] = {}
    for word in words:
        emissions[state][word] = hmm_tagger_NLTK._outputs[state].prob(word)

initial = {}
for state in tags_nltk:
    initial[state] = hmm_tagger_NLTK._priors.prob(state)

In [ ]:
NLTK_tagger = Predictor(tags_nltk, transitions, emissions, initial)

In [ ]:
initial_df = pd.DataFrame([initial], columns=tags_nltk)
initial_df

In [ ]:
emissions_df = pd.DataFrame.from_dict(emissions)
print(sum(emissions_df.iloc[:, 0]))

In [ ]:
transitions_df = pd.DataFrame.from_dict(transitions)
print(sum(transitions_df.iloc[:, 0]))

With HMM trained with own Baum-Welch implementation

In [ ]:
with open(BW_model, 'rb') as pickle_file:
    hmm_tagger_BW = pickle.load(pickle_file)

In [ ]:
# Extract model parameters
tags_bw = hmm_tagger_BW["states"]  # Set of all possible tags
words = hmm_tagger_BW["vocab"]  # Set of all possible words

# convert probability distributions of HMM to dictionaries
transitions = {}
transition_probs = hmm_tagger_BW["transition_probs"]
for prev_idx, prev_state in enumerate(tags_bw):
    transitions[prev_state] = {}
    for next_idx, next_state in enumerate(tags_bw):
        transitions[prev_state][next_state] = transition_probs[prev_idx, next_idx]

emissions = {}
emission_probs = hmm_tagger_BW["emission_probs"]
for state_idx, state in enumerate(tags_bw):
    emissions[state] = {}
    for word_idx, word in enumerate(words):
        emissions[state][word] = emission_probs[state_idx, word_idx]

initial = {}
initial_probs = hmm_tagger_BW["initial_probs"]
for state_idx, state in enumerate(tags_bw):
    initial[state] = initial_probs[state_idx]

In [ ]:
BW_tagger = Predictor(tags_bw, transitions, emissions, initial)

In [ ]:
initial_df = pd.DataFrame([initial], columns=tags_bw)
initial_df

In [ ]:
emissions_df = pd.DataFrame.from_dict(emissions)
print(sum(emissions_df.iloc[:, 0]))

In [ ]:
transitions_df = pd.DataFrame.from_dict(transitions)
print(sum(transitions_df.iloc[:, 0]))

# Evaluate

Data for evaluation

In [ ]:
train, test = load_brown_data(data_file, split=0.8)

test_sample = test # can split test to test on a smaller sample
test_sample_sequences= [[token.get_word() for token in sentence] for sentence in test_sample]

Run prediction algorithm

In [ ]:
targets = [[token.get_pos() for token in sentence] for sentence in test_sample]

In [ ]:
predictions_nltk = [NLTK_tagger.viterbi(sequence) for sequence in test_sample_sequences]

In [ ]:
predictions_bw = [BW_tagger.viterbi(sequence) for sequence in test_sample_sequences]

## Accuracy

In [ ]:
def compute_accuracy(actuals, targets):
    """Compute accuracy of guesses comparing to target data.
    actual: list of guesses from the model ex. [['VERB', 'NOUN'], ['DET']]
    target: list values from the test/validation set
    """
    if len(actuals) != len(targets):
        return -1 # the number of actual values should match number of target values
    correct_count = 0
    total_tags = 0
    for actual_tags, target_tags in zip(actuals, targets):
        total_tags += len(actual_tags)
        if len(actual_tags) != len(target_tags):
            return -1 # the number of actual values should match number of target values
        for actual_value, target_value in zip(actual_tags, target_tags):
            if actual_value == target_value:
                correct_count += 1
    
    return correct_count/total_tags

Evaluate NLTK Trained model

In [ ]:
print(compute_accuracy(predictions_nltk, targets))

Evaluate HMM trained with own Baum-Welch algorithm

In [ ]:
print(compute_accuracy(predictions_bw, targets))

## Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def create_hmm_confusion_matrix(true_states, predicted_states, title="HMM States Confusion Matrix", 
                               state_names=None, normalize=True):
    """
    Create and visualize a confusion matrix for HMM state predictions.
    
    Parameters:
    -----------
    true_states : list or array
        The true hidden state sequences from your labeled test data
    predicted_states : list or array
        The Viterbi-decoded state sequences from your model
    title : str
        Title for the confusion matrix plot
    state_names : list, optional
        Names of the hidden states (if they have semantic meaning)
    normalize : bool
        Whether to normalize the confusion matrix by row
    
    Returns:
    --------
    cm : numpy.ndarray
        The confusion matrix
    """
    # Flatten sequences if they're nested
    if isinstance(true_states[0], (list, np.ndarray)):
        true_flat = np.concatenate(true_states)
        pred_flat = np.concatenate(predicted_states)
    else:
        true_flat = np.array(true_states)
        pred_flat = np.array(predicted_states)
    
    # Get unique states
    unique_states = sorted(set(np.concatenate([np.unique(true_flat), np.unique(pred_flat)])))
    
    # Create confusion matrix
    cm = confusion_matrix(true_flat, pred_flat, labels=unique_states)
    
    # Normalize if requested
    if normalize:
        cm_plot = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
    else:
        cm_plot = cm
        fmt = 'd'
    
    # Visualization
    plt.figure(figsize=(10, 8))
    if state_names is None:
        state_names = [f"State {i}" for i in unique_states]
    
    sns.heatmap(cm_plot, annot=True, fmt=fmt, cmap='BuPu',
                xticklabels=state_names, yticklabels=state_names)
    plt.xlabel('Predicted States')
    plt.ylabel('True States')
    plt.title(title)
    plt.tight_layout()
    plt.show()
    
    return cm

In [ ]:
# - targets: list of true state sequences from your test data
# - predictions_nltk: predictions from your supervised model
# - predictions_bw: predictions from your unsupervised model

# Confusion matrices
supervised_cm = create_hmm_confusion_matrix(
    targets, 
    predictions_nltk,
    state_names=tags_nltk,
    title="Supervised HMM Confusion Matrix"
)

unsupervised_cm = create_hmm_confusion_matrix(
    targets, 
    predictions_bw,
    state_names=tags_bw,
    title="Unsupervised HMM Confusion Matrix"
)

#  Detailed error analysis
def analyze_errors(cm, true_states, pred_states):
    """Analyze which sequences have the most errors"""  
    # Calculate errors per sequence
    errors = []
    for true_seq, pred_seq in zip(true_states, pred_states):
        errors.append(sum(t != p for t, p in zip(true_seq, pred_seq)))
    
    # Find most problematic sequences
    sorted_indices = np.argsort(errors)[::-1]
    
    print("Top 5 most problematic sequences:")
    for i in sorted_indices[:5]:
        print(f"Sequence {i}: {errors[i]} errors")
        print(f"  True:      {[s for s in true_states[i]]}")
        print(f"  Predicted: {[s for s in pred_states[i]]}")
        sequence = " ".join(test_sample[i])
        print(f"{sequence}")
        print()